## Module for building Henderson's Mixed Model Equations for single trait including marker information and polygenic effect



### Data set to test the module

In [1]:
using DataFrames
using Distributions

In [2]:
FILE = "../../data/small.ped";

In [3]:
Animal = ["S1","D1","O1","O3"]
y = [100.0, 50.0, 150.0, 40.0];
df = DataFrame(Animal=Animal,y=y);

In [4]:
d = Binomial(2,0.5)
nObs     = 4
nMarkers = 5
M        = float(rand(d,(nObs,nMarkers)));

In [5]:
df=[df DataFrame(M)]

,Animal,y,x1,x2,x3,x4,x5
1,S1,100.0,2.0,1.0,1.0,2.0,0.0
2,D1,50.0,2.0,0.0,1.0,2.0,2.0
3,O1,150.0,2.0,0.0,0.0,0.0,0.0
4,O3,40.0,1.0,0.0,0.0,0.0,1.0


### <font color="red"> Run module</font>

In [6]:
MODULE_PATH="../../Module/MME.jl";

In [7]:
include(MODULE_PATH);

In [8]:
ped = PedModule.mkPed(FILE);

In [9]:
mme = MMEModule.initMME("y = intercept + Animal")
varg=1.0
vare=1.0
λ1 = vare/varg
G = reshape([λ1],1,1)
MMEModule.setAsRandom(mme,"Animal",ped,G)

In [11]:
MMEModule.getMME(mme,df)

LoadError: LoadError: BoundsError
while loading In[11], in expression starting on line 1

In [ ]:
resG = MMEModule.getSolG(mme,df)

### Gibbs

In [ ]:
sol = fill(0.0,size(mme.mmeLhs,1));

In [ ]:
res1=MMEModule.Gibbs(mme.mmeLhs,sol,mme.mmeRhs,50000,outFreq=10000)

### add marker information

In [16]:
MMEModule.addMarkers(mme,df[:,3:7]);

In [17]:
mme.M

4x5 Array{Float64,2}:
 2.0  1.0  1.0  2.0  0.0
 2.0  0.0  1.0  2.0  2.0
 2.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0

In [46]:
function sampleMCMC(nIter,mme,df;outFreq=100)
    vRes = 1.0
    vEff = 1.0   
    if size(mme.mmeRhs)==() 
        MMEModule.getMME(mme,df)
    end
    p = size(mme.mmeLhs,1)
    sol = fill(0.0,p)
    solMean = fill(0.0,p)

    α  = zeros(Float64,size(mme.M,2))
    meanAlpha = zeros(Float64,size(mme.M,2))
    mArray = MMEModule.get_column_ref(mme.M)
    mpm = [dot(mme.M[:,i],mme.M[:,i]) for i=1:size(mme.M,2)]   
    ycorr = mme.ySparse 
    
    for iter=1:nIter
        #sample non-marker part
        mme.ySparse = ycorr + mme.X*sol
        mme.mmeRhs = mme.X'mme.ySparse
        MMEModule.Gibbs(mme.mmeLhs,sol,mme.mmeRhs)
        ycorr = mme.ySparse - mme.X*sol

        solMean += (sol - solMean)/iter
        
        #sample marker
        MMEModule.sample_effects_ycorr!(mme.M,mArray,mpm,vec(ycorr),α,meanAlpha,vRes,vEff,iter)
        
        if iter%outFreq==0
            println("at sample: ",iter)
        end
    end
    output = Dict()
    output["posteriorMeanLocationParms"] = solMean
    output["posteriorMeanMarkerEffects"] = meanAlpha
    return output
end

sampleMCMC (generic function with 1 method)

In [47]:
sampleMCMC(1000,mme,df,outFreq=10)

at sample: 10

Dict{Any,Any} with 1 entry:
  "posteriorMeanLocationP… => [109.08282489786498,-9.627542044374989,-60.001480…


at sample: 20
at sample: 30
at sample: 40
at sample: 50
at sample: 60
at sample: 70
at sample: 80
at sample: 90
at sample: 100
at sample: 110
at sample: 120
at sample: 130
at sample: 140
at sample: 150
at sample: 160
at sample: 170
at sample: 180
at sample: 190
at sample: 200
at sample: 210
at sample: 220
at sample: 230
at sample: 240
at sample: 250
at sample: 260
at sample: 270
at sample: 280
at sample: 290
at sample: 300
at sample: 310
at sample: 320
at sample: 330
at sample: 340
at sample: 350
at sample: 360
at sample: 370
at sample: 380
at sample: 390
at sample: 400
at sample: 410
at sample: 420
at sample: 430
at sample: 440
at sample: 450
at sample: 460
at sample: 470
at sample: 480
at sample: 490
at sample: 500
at sample: 510
at sample: 520
at sample: 530
at sample: 540
at sample: 550
at sample: 560
at sample: 570
at sample: 580
at sample: 590
at sample: 600
at sample: 610
at sample: 620
at sample: 630
at sample: 640
at sample: 650
at sample: 660
at sample: 670
at sample: 680
at